In [ ]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=eedc9256cd0fadbe84fb0d6809e873a447d09e7af0efd4fb46ccd61e103dccf4
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [ ]:
!pip install jsonlines


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install simcse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 79.3 MB/s eta 0:00:00
  Created wheel for simcse: filename=simcse-0.4-py3-none-any.whl size=15034 sha256=c338dbca74232c582faa4d035d5afdf39508f320b785b34d6068e13c962e06a0
  Stored in directory: /root/.cache/pip/wheels/80/0f/b7/10f417b839634de949621bf112ed37c5f924d9b83146d79084
Successfully built simcse
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beh

In [ ]:
from modulefinder import packagePathMap
from unittest import result
from click import pass_obj
from sentence_transformers import SentenceTransformer
from simcse import SimCSE
from transformers import AutoTokenizer, AutoModel
from sklearn.feature_extraction.text import TfidfVectorizer
import json
import jsonlines
import numpy as np
import math
import os
import argparse
import statistics
import codecs
import json
import csv

##Paper Parser

In [ ]:
class get_doc:
    def __init__(self, paper_id, metadata, title, abstract, pred_labels_truncated, pred_labels):
      self.paper_id = paper_id
      self.metadata = metadata
      self.title = title
      self.abstract = abstract
      self.pred_labels_truncated = pred_labels_truncated
      self.pred_labels = pred_labels

docs = {}

with jsonlines.open('/content/drive/MyDrive/data/abstracts-csfcube-preds.jsonl') as doc:
  for section in doc:
    docs[section['paper_id']] = get_doc(section['paper_id'], section['metadata'], section['title'], section['abstract'], section['pred_labels_truncated'], section['pred_labels'])
print(len(docs))

4207


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def get_data_by_facet(pid, facet):
  predLabels = {'background' : ['background_label', 'objective_label'], 'method' : ['method_label'], 'result' : ['result_label']}
  doc = docs[pid];
  temp = list()
  for pred_index in range(len(doc.pred_labels)):
    if doc.pred_labels[pred_index] in predLabels[facet]:
      temp.append(doc.abstract[pred_index])

  return temp

In [ ]:
def get_bag_of_words(corpus):
  tf = TfidfVectorizer().fit([corpus])
  return tf

def get_tf_idf_doc(bag_of_words, query):
    return bag_of_words.transform([query]).toarray()[0]
  
def get_bert_nli_embedding(sentence):
    return model_sent_bert_nli.encode(sentence)

def get_bert_pp_embedding(sentence):
    return model_sent_bert_pp.encode(sentence)

def get_all_mpnet_base_v2_embedding(sentence):
    return model_all_mpnet_base_v2.encode(sentence)
  
def get_specter_embedding(sentence):
    inputs = tokenizer_specter(sentence, padding=True, truncation=True, return_tensors="pt", max_length=65536)
    return model_specter(**inputs).last_hidden_state[:, 0, :]


In [ ]:
f = open('/content/drive/MyDrive/data/test-pid2anns-csfcube-background.json')
background_query = json.load(f)
background_query_ids = list(background_query.keys())
print(background_query_ids)

f = open('/content/drive/MyDrive/data/test-pid2anns-csfcube-method.json')
method_query = json.load(f)
method_query_ids = list(method_query.keys())
print(method_query_ids)

f = open('/content/drive/MyDrive/data/test-pid2anns-csfcube-result.json')
result_query = json.load(f)
result_query_ids = list(result_query.keys())
print(result_query_ids)

['10014168', '10695055', '5764728', '51977123', '929877', '3264891', '1936997', '11844559', '8781666', '1587', '189897839', '6173686', '7898033', '52194540', '1791179', '6431039']
['1198964', '11310392', '10010426', '53080736', '53082542', '1791179', '2468783', '80628431', '189897839', '52194540', '929877', '1936997', '102353905', '5052952', '13949438', '174799296', '5270848']
['1198964', '10052042', '11629674', '1587', '174799296', '1306065', '53080736', '2090262', '2865563', '11844559', '2468783', '5052952', '2360770', '3264891', '6431039', '80628431', '8781666']


In [ ]:
print(get_data_by_facet('143814895', 'method'))

['We show that the actual diachronic frequency distributions and those expected under the neutral model do not coincide and conclude that in this case the neutral model does not provide an adequate description of change in ceramic decoration.']


In [ ]:
def cosine_similarity(doc1, doc2):
  a = np.asarray(doc1)
  b = np.asarray(doc2)
  cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))

  return cos_sim

In [ ]:
print(cosine_similarity([1, 0, 5], [2, 1, 0]))

0.17541160386140586


## Corpus Creation

In [ ]:
corpus_list = []
for doc in docs:
    doc_abstract = docs[doc].abstract
    corpus_list = corpus_list + doc_abstract
print(corpus_list[0])

corpus = " ".join(corpus_list)

Many parallel and distributed message-passing programs are written in a parametric way over available resources, in particular the number of nodes and their topologies, so that a single parallel program can scale over different environments.


Tf-Idf Computation

In [ ]:
bag_of_words = get_bag_of_words(corpus)
print(type(bag_of_words))

<class 'sklearn.feature_extraction.text.TfidfVectorizer'>


## TF-IDF

In [ ]:
# tf_idf all
d = dict()
err_pids = list()
i = 0
for doc in docs:
    i += 1
    if i % 1000 == 0:
        print(i)
    try:
        doc_abstract = docs[doc].abstract
        if len(doc) == 0:
            d[doc] = list()
            continue
        abstract_tf_idf = np.array(get_tf_idf_doc(bag_of_words, " ".join(doc_abstract))).tolist()
        if i % 1000 == 0:
            print(abstract_tf_idf)
            # ans = 0
            # for j in abstract_tf_idf:
            #     if j > 0:
            #         ans = ans + 1
            # print("ans =", ans)
        d[doc] = abstract_tf_idf
    except:
        err_pids.append(doc)

print(f'len(err_pids) = {len(err_pids)}')
print(err_pids)
with open("/content/drive/MyDrive/Results/tf_idf/all.json", "w") as outfile:
    json.dump(d, outfile)
d = json.load(open('/content/drive/MyDrive/Results/tf_idf/all.json'),
              parse_float=lambda x: round(float(x), 6))
with open('/content/drive/MyDrive/Results/tf_idf/all.json', "w") as outfile:
    json.dump(d, outfile)

1000
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

FileNotFoundError: ignored

In [ ]:
# tf_idf queries
for i, ids in enumerate([background_query_ids, method_query_ids, result_query_ids]):
    d = dict()
    err_pids = list()
    s = ''
    if i == 0:
        s = 'background'
    elif i == 1:
        s = 'method'
    elif i == 2:
        s = 'result'
    for pid in ids:
        try:
            doc = get_data_by_facet(pid, s)
            if len(doc) == 0:
                d[pid] = list()
                continue
            
            query_tfidf_array = np.array(get_tf_idf_doc(bag_of_words, " ".join(doc))).tolist()
            ans = 0
            for j in query_tfidf_array:
                if j > 0:
                    ans = ans + 1
            print("ans =", ans)
            d[pid] = query_tfidf_array
        except:
            err_pids.append(pid)
    print(f'len(err_pids), {s} = {len(err_pids)}')
    print(err_pids)
    with open(f'/content/drive/MyDrive/Results/tf_idf/{s}.json', "w") as outfile:
        json.dump(d, outfile)
    d = json.load(open(f'/content/drive/MyDrive/Results/tf_idf/{s}.json'),
                parse_float=lambda x: round(float(x), 6))
    with open(f'/content/drive/MyDrive/Results/tf_idf/{s}.json', "w") as outfile:
        json.dump(d, outfile)

ans = 23
ans = 39
ans = 76
ans = 55
ans = 46
ans = 40
ans = 36
ans = 14
ans = 44
ans = 21
ans = 51
ans = 18
ans = 61
ans = 74
ans = 31
ans = 27
len(err_pids), background = 0
[]
ans = 59
ans = 77
ans = 41
ans = 31
ans = 48
ans = 29
ans = 42
ans = 43
ans = 30
ans = 35
ans = 29
ans = 26
ans = 41
ans = 76
ans = 21
ans = 44
ans = 21
len(err_pids), method = 0
[]
ans = 30
ans = 27
ans = 37
ans = 16
ans = 33
ans = 42
ans = 53
ans = 40
ans = 47
ans = 49
ans = 25
ans = 31
ans = 38
ans = 25
ans = 50
ans = 49
ans = 32
len(err_pids), result = 0
[]


Cosine Similarity

In [ ]:
def get_relevant_docs(qid, q_type, n, model_type, data, corpus, queries):
    # corpus = json.load(open(f'/content/drive/MyDrive/Results/{model_type}/all.json')) 
    # queries = json.load(open(f'/content/drive/MyDrive/Results/{model_type}/{q_type}.json')) 
    query = queries[qid] 
    # data = json.load(open(f'/content/drive/MyDrive/data/test-pid2anns-csfcube-{q_type}.json')) 
    candidates = data[qid]['cands']
    results = dict()
    for i in candidates:
        results[i] = cosine_similarity(query, corpus[i])

    sorted_results = sorted(results.items(), key=lambda kv: (kv[1], kv[0]), reverse=True)

    # sorted_results = sorted(results.items(), key=lambda x:x[1])
    # d1= dict(list(d.items())[0: N]) 
    final = sorted_results[0:n]
    return final, results, sorted_results

## Metric Functions

In [ ]:
# Precision Calculation
def r_precision(r):
    r = np.asarray(r) != 0
    z = r.nonzero()[0]
    if not z.size:
        return 0.
    return np.mean(r[:z[-1] + 1])

In [ ]:
#P@K
# r:- list of non-relevant and relevant queries
# k:- K
def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    return np.mean(r)

In [ ]:
def average_precision(r):
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

In [ ]:
def mean_average_precision(rs):
    return np.mean([average_precision(r) for r in rs])

In [ ]:
# def list_former(qid, type, n):
#     corpus = json.load(open('/content/drive/MyDrive/Results/tf_idf/all.json')) 
#     queries = json.load(open(f'/content/drive/MyDrive/Results/tf_idf/{type}.json')) 
#     query = queries[qid] 
#     data = json.load(open('/content/drive/MyDrive/data/test-pid2anns-csfcube-background.json')) 
#     candidates = data[qid]['cands']
#     results = dict()
#     for i in candidates:
#         results[i] = cosine_similarity(query, corpus[i])
#     return results

In [ ]:
# def form_list(retreived, delta):
#   r = []
#   for qid in retreived:
#     if retreived[qid] >= delta:
#       r.append(1)
#     else:
#       r.append(0)
#   return r
# def form_list_weighted(retreived):
#   r = []
#   for qid in retreived:
#     r.append(retreived[qid]//0.25)
#   return r

In [ ]:
# sorted_results = sorted(retreived.items(), key=lambda kv: (kv[1], kv[0]), reverse=True)
# r = []
# r = form_list(retreived, 0.3)
# print(r_precision(r))
# print(precision_at_k(r, 20))
# print(average_precision(r))
# ans = []
# ans.append(r)
# print(mean_average_precision(ans))

In [ ]:
# r = form_list_weighted(retreived)
# print(r_precision(r))

# print(precision_at_k(r, 20))
# print(average_precision(r))
# ans = []
# ans.append(r)
# print(mean_average_precision(ans))

NDCG

In [ ]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, k, method=0):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max



In [ ]:
print(ndcg_at_k([1, 1, 1, 1, 1, 2, 1, 2, 1, 1], 10))

0.8235833168864323


In [ ]:

def recall_at_k(ranked_rel, atk, max_total_relevant):
    # print(ranked_rel)
    total_relevant = sum(ranked_rel)
    # total_relevant = min(max_total_relevant, len(ranked_rel))
    total_relevant = min(max_total_relevant, total_relevant)
    relatk = sum(ranked_rel[:atk])
    # print(relatk, total_relevant)
    if total_relevant > 0:
        recall_atk = float(relatk)/total_relevant
    else:
        recall_atk = 0.0
    return recall_atk

# def average_recall(r, max_total, n):
#     r = np.asarray(r) != 0
#     out = [recall_at_k(r, k + 1, max_total) for k in range(r.size) if r[k]]
#     print(len(out), n)
#     if not out:
#         return 0.0
#     return out[min(len(out), n)-1]

# def mean_average_recall(rs):
#     return np.mean([average_recall(r[0], r[1], r[2]) for r in rs])

In [ ]:
# print(mean_average_precision([[1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0], [1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0]]))

0.6393671419616797


In [ ]:
# def evaluation_all(q_type, n, model_type, ann_type):
#   queries = json.load(open(f'/content/drive/MyDrive/data/test-pid2anns-csfcube-{q_type}.json'))
#   rp = 0
#   p20 = 0
#   r20 = 0
#   ndcg100 = 0
#   ndcg20 = 0
#   ndcg_20 = 0
#   rs = []
#   rc = []
#   for query in queries:
#     print(query)
#     # print(queries[query].keys())
#     sorted_k, unsorted_all, sorted_all = get_relevant_docs(query, q_type, n, model_type)
#     rel_adju = queries[query][ann_type]
#     candidates = queries[query]['cands']
#     score = dict()
#     for i in range(len(candidates)):
#       score[candidates[i]] = rel_adju[i]
#     precision_list = []
#     for doc_id in unsorted_all.keys():
#       precision_list.append(1 if score[doc_id] >= 2 else 0)
#     rp += r_precision(precision_list)
#     sorted_rel = []
#     for tup in sorted_k:
#       sorted_rel.append(1 if score[tup[0]] >= 2 else 0)
#     rs.append(sorted_rel)
#     r = np.asarray(rel_adju)
#     z = r.nonzero()[0]
#     t = z.size
#     sorted_rel2 = []
#     # print(len(score), score)
#     for i in sorted_all:
#       sorted_rel2.append(1 if score[i[0]] >= 2 else 0)
#     rc.append((sorted_rel2, len(rel_adju), n))
#     cnt = 0;
#     for i in rel_adju:
#       if i >= 2:
#         cnt += 1
#     r20 += recall_at_k(sorted_rel2, n, cnt)
#     # print(sorted_rel2, sorted_all)
#     p = t/len(rel_adju)
#     # k = p * len(rel_adju) # ndcg100
#     # k20 = k/5             # ndcg20
#     sorted = []
#     for i in sorted_all:
#       sorted.append(score[i[0]])
#     ndcg100 += ndcg_at_k(sorted,len(rel_adju))
#     ndcg20 += ndcg_at_k(sorted, int(0.2*len(rel_adju)))
#     ndcg_20 += ndcg_at_k(sorted, 20)

#   rp = rp/len(queries)
#   # print(rs)
#   p20 = mean_average_precision(rs)
#   ndcg100 = ndcg100/len(queries)
#   ndcg20 = ndcg20/len(queries)
#   ndcg_20 /= len(queries)
#   r20 /= len(queries)
#   return rp, p20, r20, ndcg100, ndcg20, ndcg_20

In [ ]:
# print(evaluation_all("background", 20, "tf_idf", "relevance_adju"))

10014168
10695055
5764728
51977123
929877
3264891
1936997
11844559
8781666
1587
189897839
6173686
7898033
52194540
1791179
6431039
(0.13661103284394546, 0.494374771615786, 0.3848256930977519, 0.7346210264547696, 0.4921747233089462, 0.48192747439747863)


In [ ]:
# print(evaluation_all("method", 20, "tf_idf"))

1198964
11310392
10010426
53080736
53082542
1791179
2468783
80628431
189897839
52194540
929877
1936997
102353905
5052952
13949438
174799296
5270848
[[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1], [0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1], [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1], [1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1], [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,

In [ ]:
# print(evaluation_all("result", 20, "tf_idf", "relevance_adju"))

1198964
10052042
11629674
1587
174799296
1306065
53080736
2090262
2865563
11844559
2468783
5052952
2360770
3264891
6431039
80628431
8781666
(0.0880586354048245, 0.35067429955376467, 0.25663682722506254, 0.6040823229209368, 0.3296320802937453, 0.3192982995205753)


In [ ]:
# pr1, p20_1, r20_1, ndcg_1, ndcg_20_1, ndcgat20_1 = evaluation_all("result", 20, "bert_nli", "relevance_max")
# # pr2, p20_2, r20_2, ndcg_2, ndcg_20_2, ndcgat20_2 = evaluation_all("result", 20, "bert_nli", "relevance_ann4")
# # pr3, p20_3, r20_3, ndcg_3, ndcg_20_3, ndcgat20_3 = evaluation_all("result", 20, "bert_nli", "relevance_ann2")
# pr4, p20_4, r20_4, ndcg_4, ndcg_20_4, ndcgat20_4 = evaluation_all("result", 20, "bert_nli", "relevance_min")
# pr5, p20_5, r20_5, ndcg_5, ndcg_20_5, ndcgat20_5 = evaluation_all("result", 20, "bert_nli", "relevance_adju")

# print(np.mean(np.array([pr1, pr4, pr4])))
# print(np.mean(np.array([p20_1, p20_4, p20_5])))
# print(np.mean(np.array([r20_1, r20_4, r20_5])))
# print(np.mean(np.array([ndcg_1, ndcg_4, ndcg_5])))
# print(np.mean(np.array([ndcg_20_1, ndcg_20_4, ndcg_20_5])))
# print(evaluation_all("result", 20, "bert_nli", "relevance_adju"))
# print(evaluation_all("result", 20, "bert_nli", "relevance_adju"))
# print(evaluation_all("result", 20, "bert_nli", "relevance_min"))

1198964
10052042
11629674
1587
174799296
1306065
53080736
2090262
2865563
11844559
2468783
5052952
2360770
3264891
6431039
80628431
8781666
(0.0880586354048245, 0.3695165599948101, 0.4166578519519696, 0.6588360002600858, 0.42400150289245425, 0.4072351074140363)


In [ ]:
# relevant_cnt = []
# for x in score.values():
#   relevant_cnt.append(x)
# relevant_cnt = (np.asarray(relevant_cnt) != 0).size
# print(relevant_cnt)
# temp = []
# for i in relevant:
#   if i[0] in score.keys():
#     temp.append(score[i[0]])

# print(recall_at_k(temp, 10, relevant_cnt))

NameError: ignored

In [ ]:
data = json.load(open('/content/drive/MyDrive/data/test-pid2anns-csfcube-background.json'))
for query in data:
  retreived = list_former("10695055", "background", len(data["10695055"]['cands']))
candidates = data['10695055']['cands']
rel_max = data['10695055']['relevance_max']
score = dict()
for i in range(len(candidates)):
  score[candidates[i]] = rel_max[i]

## Bert NLI and PP

In [ ]:
model_sent_bert_nli = SentenceTransformer('nli-roberta-base-v2')

In [ ]:
model_sent_bert_pp = SentenceTransformer('paraphrase-TinyBERT-L6-v2')

In [ ]:
model_all_mpnet_base_v2 = SentenceTransformer('all-mpnet-base-v2')

## BERT NLI

In [ ]:
# bert_nli all
d = dict()
err_pids = list()
i = 0
for doc in docs:
    try:
        do = docs[doc].abstract
        if len(do) == 0:
            d[pid] = []
            continue
        bert_nli_vectors = np.array(get_bert_nli_embedding(" ".join(do))).tolist()
        d[pid] = bert_nli_vectors
    except:
        err_pids.append(pid)
print(f'len(err_pids) = {len(err_pids)}')
print(err_pids)
with open("/content/drive/MyDrive/Results/bert_nli/all.json", "w") as outfile:
    json.dump(d, outfile)
d = json.load(open('/content/drive/MyDrive/Results/bert_nli/all.json'),
              parse_float=lambda x: round(float(x), 6))
with open('/content/drive/MyDrive/Results/bert_nli/all.json', "w") as outfile:
    json.dump(d, outfile)

len(err_pids) = 0
[]


In [ ]:
# bert_nli queries
for i, ids in enumerate([background_query_ids, method_query_ids, result_query_ids]):
    d = dict()
    err_pids = list()
    s = ''
    if i == 0:
        s = 'background'
    elif i == 1:
        s = 'method'
    elif i == 2:
        s = 'result'
    for pid in ids:
        try:
            doc = get_data_by_facet(pid, s)
            if len(doc) == 0:
                d[pid] = []
                continue
            bert_nli_query_vectors = np.array(get_bert_nli_embedding(" ".join(doc))).tolist()
            d[pid] = bert_nli_query_vectors
        except:
            err_pids.append(pid)
    print(f'len(err_pids), {s} = {len(err_pids)}')
    print(err_pids)
    with open(f'/content/drive/MyDrive/Results/bert_nli/{s}.json', "w") as outfile:
        json.dump(d, outfile)
    d = json.load(open(f'/content/drive/MyDrive/Results/bert_nli/{s}.json'),
                parse_float=lambda x: round(float(x), 6))
    with open(f'/content/drive/MyDrive/Results/bert_nli/{s}.json', "w") as outfile:
        json.dump(d, outfile)

len(err_pids), background = 0
[]
len(err_pids), method = 0
[]
len(err_pids), result = 0
[]


## METRICS

In [ ]:
facet2folds = {
    "background": {"fold1_dev": ["3264891_background", "1936997_background", "11844559_background",
                                 "52194540_background", "1791179_background", "6431039_background",
                                 "6173686_background", "7898033_background"],
                   "fold2_dev": ["5764728_background", "10014168_background", "10695055_background",
                                 "929877_background", "1587_background", "51977123_background",
                                 "8781666_background", "189897839_background"],
                   "fold1_test": ["5764728_background", "10014168_background", "10695055_background",
                                  "929877_background", "1587_background", "51977123_background",
                                  "8781666_background", "189897839_background"],
                   "fold2_test": ["3264891_background", "1936997_background", "11844559_background",
                                  "52194540_background", "1791179_background", "6431039_background",
                                  "6173686_background", "7898033_background"]},
    "method": {"fold1_dev": ["189897839_method", "1791179_method", "11310392_method", "2468783_method",
                             "13949438_method", "5270848_method", "52194540_method", "929877_method"],
               "fold2_dev": ["5052952_method", "10010426_method", "102353905_method", "174799296_method",
                             "1198964_method", "53080736_method", "1936997_method", "80628431_method",
                             "53082542_method"],
               "fold1_test": ["5052952_method", "10010426_method", "102353905_method", "174799296_method",
                              "1198964_method", "53080736_method", "1936997_method", "80628431_method",
                              "53082542_method"],
               "fold2_test": ["189897839_method", "1791179_method", "11310392_method", "2468783_method",
                              "13949438_method", "5270848_method", "52194540_method", "929877_method"]},
    "result": {"fold1_dev": ["2090262_result", "174799296_result", "11844559_result", "2468783_result",
                             "1306065_result", "5052952_result", "3264891_result", "8781666_result"],
               "fold2_dev": ["2865563_result", "10052042_result", "11629674_result", "1587_result",
                             "1198964_result", "53080736_result", "2360770_result", "80628431_result",
                             "6431039_result"],
               "fold1_test": ["2865563_result", "10052042_result", "11629674_result", "1587_result",
                              "1198964_result", "53080736_result", "2360770_result", "80628431_result",
                              "6431039_result"],
               "fold2_test": ["2090262_result", "174799296_result", "11844559_result", "2468783_result",
                              "1306065_result", "5052952_result", "3264891_result", "8781666_result"]},
    "all": {"fold1_dev": ["3264891_background", "1936997_background", "11844559_background",
                          "52194540_background", "1791179_background", "6431039_background",
                          "6173686_background", "7898033_background", "189897839_method",
                          "1791179_method", "11310392_method", "2468783_method", "13949438_method",
                          "5270848_method", "52194540_method", "929877_method", "2090262_result",
                          "174799296_result", "11844559_result", "2468783_result", "1306065_result",
                          "5052952_result", "3264891_result", "8781666_result"],
            "fold2_dev": ["5764728_background", "10014168_background", "10695055_background",
                          "929877_background", "1587_background", "51977123_background",
                          "8781666_background", "189897839_background", "5052952_method", "10010426_method",
                          "102353905_method", "174799296_method", "1198964_method", "53080736_method",
                          "1936997_method", "80628431_method", "53082542_method", "2865563_result",
                          "10052042_result", "11629674_result", "1587_result", "1198964_result",
                          "53080736_result", "2360770_result", "80628431_result", "6431039_result"],
            "fold1_test": ["5764728_background", "10014168_background", "10695055_background",
                           "929877_background", "1587_background", "51977123_background", "8781666_background",
                           "189897839_background", "5052952_method", "10010426_method", "102353905_method",
                           "174799296_method", "1198964_method", "53080736_method", "1936997_method",
                           "80628431_method", "53082542_method", "2865563_result", "10052042_result",
                           "11629674_result", "1587_result", "1198964_result", "53080736_result",
                           "2360770_result", "80628431_result", "6431039_result"],
            "fold2_test": ["3264891_background", "1936997_background", "11844559_background",
                           "52194540_background", "1791179_background", "6431039_background",
                           "6173686_background", "7898033_background", "189897839_method", "1791179_method",
                           "11310392_method", "2468783_method", "13949438_method", "5270848_method",
                           "52194540_method", "929877_method", "2090262_result", "174799296_result",
                           "11844559_result", "2468783_result", "1306065_result", "5052952_result",
                           "3264891_result", "8781666_result"]
            }
}

In [ ]:
def read_facet_specific_relevances(data_path, run_path, dataset, facet, method_name):
    """
    Read the gold data and the model rankings and the relevances for the
    model.
    :param data_path: string; directory with gold citations for test pids and rankings
        from baseline methods in subdirectories.
    :param run_path: string; directory with ranked candidates for baselines a subdir of
        data_path else is a model run.
    :param method_name: string; method with which ranks were created.
    :param dataset: string; eval dataset.
    :param facet: string; facet for eval.
    :return: qpid2rankedcand_relevances: dict('qpid_facet': [relevances]);
        candidate gold relevances for the candidates in order ranked by the
        model.
    """
    gold_fname = os.path.join(data_path, 'test-pid2anns-{:s}-{:s}.json'.format(dataset, facet))
    ranked_fname = os.path.join(run_path, method_name, 'test-pid2pool-{:s}-{:s}-{:s}-ranked.json'.
                                format(dataset, method_name, facet))
    # Load gold test data (citations).
    with codecs.open(gold_fname, 'r', 'utf-8') as fp:
        pid2pool_source = json.load(fp)
        num_query = len(pid2pool_source)
        print('Gold query pids: {:d}'.format(num_query))
        pid2rels_gold = {}
        for qpid, pool_rel in pid2pool_source.items():
            pool = pool_rel['cands']
            cands_rels = pool_rel['relevance_adju']
            pid2rels_gold['{:s}_{:s}'.format(qpid, facet)] = dict([(pid, rel) for pid, rel in zip(pool, cands_rels)])
    # Load ranked predictions on test data with methods.
    with codecs.open(ranked_fname, 'r', 'utf-8') as fp:
        pid2ranks = json.load(fp)
        print('Valid ranked query pids: {:d}'.format(len(pid2ranks)))
        qpid2rankedcand_relevances = {}
        for qpid, citranks in pid2ranks.items():
            candpids = [pid_score[0] for pid_score in citranks]
            cand_relevances = [pid2rels_gold['{:s}_{:s}'.format(qpid, facet)][pid] for pid in candpids]
            qpid2rankedcand_relevances['{:s}_{:s}'.format(qpid, facet)] = cand_relevances
    return qpid2rankedcand_relevances


def read_all_facet_relevances(data_path, run_path, dataset, method_name, facets):
    """
    Read the gold data and the model rankings and the relevances for the
    model.
    :param data_path: string; directory with gold citations for test pids and rankings
        from baseline methods in subdirectories.
    :param run_path: string; directory with ranked candidates for baselines a subdir of
        data_path else is a model run.
    :param method_name: string; method with which ranks were created.
    :param dataset: string; eval dataset.
    :param facets: list(string); what facets to read/what counts as "all".
    :return: qpid2rankedcand_relevances: dict('qpid_facet': [relevances]);
        candidate gold relevances for the candidates in order ranked by the
        model.
    """
    qpid2rankedcand_relevances = {}
    for facet in facets:
        print('Reading facet: {:s}'.format(facet))
        gold_fname = os.path.join(data_path, 'test-pid2anns-{:s}-{:s}.json'.format(dataset, facet))
        ranked_fname = os.path.join(run_path, method_name, 'test-pid2pool-{:s}-{:s}-{:s}-ranked.json'.
                                    format(dataset, method_name, facet))
        # Load gold test data (citations).
        with codecs.open(gold_fname, 'r', 'utf-8') as fp:
            pid2pool_source = json.load(fp)
            num_query = len(pid2pool_source)
            print('Gold query pids: {:d}'.format(num_query))
            pid2rels_gold = {}
            for qpid, pool_rel in pid2pool_source.items():
                pool = pool_rel['cands']
                cands_rels = pool_rel['relevance_adju']
                pid2rels_gold['{:s}_{:s}'.format(qpid, facet)] = \
                    dict([(pid, rel) for pid, rel in zip(pool, cands_rels)])
        # Load ranked predictions on test data with methods.
        with codecs.open(ranked_fname, 'r', 'utf-8') as fp:
            pid2ranks = json.load(fp)
            print('Valid ranked query pids: {:d}'.format(len(pid2ranks)))
            for qpid, citranks in pid2ranks.items():
                candpids = [pid_score[0] for pid_score in citranks]
                cand_relevances = [pid2rels_gold['{:s}_{:s}'.format(qpid, facet)][pid] for pid in candpids]
                qpid2rankedcand_relevances['{:s}_{:s}'.format(qpid, facet)] = cand_relevances
    print('Total queries: {:d}'.format(len(qpid2rankedcand_relevances)))
    return qpid2rankedcand_relevances

In [ ]:
def compute_metrics(ranked_judgements, pr_atk, threshold_grade):
    """
    Given the ranked judgements compute the metrics for a query.
    :param ranked_judgements: list(int); graded or binary relevances in rank order.
    :param pr_atk: int; the @K value to use for computing precision and recall.
    :param threshold_grade: int; Assuming 0-3 graded relevances, threshold at some point
        and convert graded to binary relevance.
    :return:
    """
    graded_judgements = ranked_judgements
    ranked_judgements = [1 if rel >= threshold_grade else 0 for rel in graded_judgements]
    # Use the full set of candidate not the pr_atk.
    ndcg = ndcg_at_k(graded_judgements, len(ranked_judgements))
    ndcg_pr = ndcg_at_k(graded_judgements, int(0.20*len(ranked_judgements)))
    ndcg_20 = ndcg_at_k(graded_judgements, 20)
    max_total_relevant = sum(ranked_judgements)
    recall = recall_at_k(ranked_rel=ranked_judgements,
                         atk=pr_atk, max_total_relevant=max_total_relevant)
    precision = precision_at_k(r=ranked_judgements, k=pr_atk)
    precision_r = r_precision(r=ranked_judgements)
    av_precision = average_precision(r=ranked_judgements)
    metrics = {
        'recall': float(recall),
        'precision': float(precision),
        'r_precision': float(precision_r),
        'av_precision': float(av_precision),
        'ndcg': ndcg,
        'ndcg@20': ndcg_20,
        'ndcg%20': ndcg_pr
    }
    return metrics

In [ ]:
def aggregate_metrics_crossval(query_metrics, split_str, facet_str):
    """
    Given metrics over individual queries aggregate over different
    queries.
    :param query_metrics: dict(query_id: metrics_dict from compute_metrics)
    :param split_str: string; {dev, test}
    :param facet_str: string; {background, method, result}
    :return:
    """
    aggmetrics = {
        'precision': [],
        'recall': [],
        'r_precision': [],
        'mean_av_precision': [],
        'ndcg': [],
        'ndcg@20': [],
        'ndcg%20': []
    }
    # For dev only use a part of the fold - using both makes it identical to test.
    if split_str == 'dev':
        folds = ['fold1_{:s}'.format(split_str)]
    elif split_str == 'test':
        folds = ['fold1_{:s}'.format(split_str), 'fold2_{:s}'.format(split_str)]
    for fold_str in folds:
        fold_pids = facet2folds[facet_str][fold_str]
        precision, recall, f1, av_precision, mrr, ndcg, r_precision = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
        ndcg_20, ndcg_pr = 0.0, 0.0
        for query_id in fold_pids:
            # Aggregate across paper types in the fold.
            metrics = query_metrics[query_id]
            # Aggregate across all papers in the fold
            precision += metrics['precision']
            recall += metrics['recall']
            av_precision += metrics['av_precision']
            r_precision += metrics['r_precision']
            ndcg += metrics['ndcg']
            ndcg_20 += metrics['ndcg@20']
            ndcg_pr += metrics['ndcg%20']
        # Average all folds
        num_queries = len(fold_pids)
        precision, recall, f1 = precision/num_queries, recall/num_queries, f1/num_queries
        av_precision = av_precision/num_queries
        mrr, ndcg, r_precision = mrr/num_queries, ndcg/num_queries, r_precision/num_queries
        ndcg_20, ndcg_pr = ndcg_20/num_queries, ndcg_pr/num_queries
        # Save the averaged metric for every fold.
        aggmetrics['precision'].append(precision)
        aggmetrics['recall'].append(recall)
        aggmetrics['r_precision'].append(r_precision)
        aggmetrics['mean_av_precision'].append(av_precision)
        aggmetrics['ndcg'].append(ndcg)
        aggmetrics['ndcg@20'].append(ndcg_20)
        aggmetrics['ndcg%20'].append(ndcg_pr)

    aggmetrics = {
        'precision': statistics.mean(aggmetrics['precision']),
        'recall': statistics.mean(aggmetrics['recall']),
        'r_precision': statistics.mean(aggmetrics['r_precision']),
        'mean_av_precision': statistics.mean(aggmetrics['mean_av_precision']),
        'ndcg': statistics.mean(aggmetrics['ndcg']),
        'ndcg@20': statistics.mean(aggmetrics['ndcg@20']),
        'ndcg%20': statistics.mean(aggmetrics['ndcg%20'])
    }
    return aggmetrics


In [ ]:
def graded_eval_pool_rerank(data_path, run_path, method_name, dataset, facet, split, ATK):
    """
    Evaluate the re-ranked pool for the faceted data. Anns use graded relevance scores.
    :param data_path: string; directory with gold citations for test pids and rankings
        from baseline methods in subdirectories.
    :param run_path: string; directory with ranked candidates for baselines a subdir of
        data_path else is a model run.
    :param method_name: string; method with which ranks were created.
    :param dataset: string; eval dataset.
    :param facet: string; facet for eval.
    :param split: strong; {dev, test}
    :return:
    """
    print(f'EVAL SPLIT: {split}')
    if facet == 'all':
        qpid2rankedcand_relevances = read_all_facet_relevances(data_path=data_path, run_path=run_path,
                                                               dataset=dataset, method_name=method_name,
                                                               facets=['background', 'method', 'result'])
    else:
        qpid2rankedcand_relevances = read_facet_specific_relevances(data_path=data_path, run_path=run_path,
                                                                    dataset=dataset, facet=facet,
                                                                    method_name=method_name)
    # Go over test papers and compute metrics.
    all_metrics = {}
    num_cands = 0.0
    num_queries = 0.0
    print('Precision and recall at rank: {:d}'.format(ATK))
    for qpid_facet, qranked_judgements in qpid2rankedcand_relevances.items():
        all_metrics[qpid_facet] = compute_metrics(qranked_judgements, pr_atk=ATK,
                                                  threshold_grade=2)
        num_cands += len(qranked_judgements)
        num_queries += 1
    aggmetrics = aggregate_metrics_crossval(query_metrics=all_metrics, facet_str=facet, split_str=split)
    print('Total queries: {:d}; Total candidates: {:d}'.format(int(num_queries), int(num_cands)))
    print('R-Precision: {:.4f}'.format(aggmetrics['r_precision']))
    print('Precision@{:d}: {:.4f}'.format(ATK, aggmetrics['precision']))
    print('Recall@{:d}: {:.4f}'.format(ATK, aggmetrics['recall']))
    print('NDCG: {:.4f}'.format(aggmetrics['ndcg']))
    print('NDCG@20: {:.4f}'.format(aggmetrics['ndcg@20']))
    print('NDCG%20: {:.4f}'.format(aggmetrics['ndcg%20']))
    return aggmetrics

In [ ]:
def get_metric_scores(method, facet, ATK, dataset='csfcube', data_path='/content/drive/MyDrive/data', run_path='/content/drive/MyDrive/Results'):
    if facet != 'all':
        data = json.load(open(f'{data_path}/test-pid2anns-{dataset}-{facet}.json'))
        corpus = json.load(open(f'{run_path}/{method}/all.json'))
        queries = json.load(open(f'{run_path}/{method}/{facet}.json'))
        d = dict()
        for qid in data:
          query = queries[qid]
          candidates = data[qid]['cands']
          results = dict()
          for i in candidates:
              results[i] = cosine_similarity(query, corpus[i])
          sorted_results = sorted(results.items(), key=lambda kv: (kv[1], kv[0]), reverse=True)
          d[qid] = sorted_results
        with open(f'{run_path}/{method}/test-pid2pool-{dataset}-{method}-{facet}-ranked.json', 'w') as outfile:
          json.dump(d, outfile)
    aggmetrics1 = graded_eval_pool_rerank(data_path=data_path,
                          method_name=method,
                                  facet=facet, dataset=dataset,
                          run_path=run_path,
                                  split='dev', ATK=ATK)
    print()
    aggmetrics2 = graded_eval_pool_rerank(data_path=data_path,
                          method_name=method,
                                  facet=facet, dataset=dataset,
                          run_path=run_path,
                                  split='test', ATK=ATK)
    
    print('\nAVERAGE METRICS')
    print('R-Precision: {:.4f}'.format((aggmetrics1['r_precision'] + aggmetrics2['r_precision']) / 2))
    print('Precision@{:d}: {:.4f}'.format(ATK, (aggmetrics1['precision'] + aggmetrics2['precision']) / 2))
    print('Recall@{:d}: {:.4f}'.format(ATK, (aggmetrics1['recall'] + aggmetrics2['recall']) / 2))
    print('NDCG: {:.4f}'.format((aggmetrics1['ndcg'] + aggmetrics2['ndcg']) / 2))
    print('NDCG@20: {:.4f}'.format((aggmetrics1['ndcg@20'] + aggmetrics2['ndcg@20']) / 2))
    print('NDCG%20: {:.4f}'.format((aggmetrics1['ndcg%20'] + aggmetrics2['ndcg%20']) / 2))

## BERT - PP

In [ ]:
# bert_pp all
d = dict()
err_pids = list()
i = 0
for doc in docs:
    # i += 1
    # if i % 50 == 0:
    #     print(i)
    try:
        do = docs[doc].abstract
        if len(do) == 0:
            d[doc] = list()
            continue
        bert_pp_array = np.array(get_bert_pp_embedding(" ".join(do))).tolist()
        d[doc] = bert_pp_array
    except:
        err_pids.append(doc)
print(f'len(err_pids) = {len(err_pids)}')
print(err_pids)
with open("/content/drive/MyDrive/Results/bert_pp/all.json", "w") as outfile:
    json.dump(d, outfile)
d = json.load(open('/content/drive/MyDrive/Results/bert_pp/all.json'),
              parse_float=lambda x: round(float(x), 6))
with open('/content/drive/MyDrive/Results/bert_pp/all.json', "w") as outfile:
    json.dump(d, outfile)

In [ ]:
# bert_pp queries
for i, ids in enumerate([background_query_ids, method_query_ids, result_query_ids]):
    d = dict()
    err_pids = list()
    s = ''
    if i == 0:
        s = 'background'
    elif i == 1:
        s = 'method'
    elif i == 2:
        s = 'result'
    for pid in ids:
        try:
            doc = get_data_by_facet(pid, s)
            if len(doc) == 0:
                d[pid] = list()
                continue
            bert_pp_array = np.array(get_bert_pp_embedding(" ".join(doc))).tolist()
            d[pid] = bert_pp_array
        except:
            err_pids.append(pid)
    print(f'len(err_pids), {s} = {len(err_pids)}')
    print(err_pids)
    with open(f'/content/drive/MyDrive/Results/bert_pp/{s}.json', "w") as outfile:
        json.dump(d, outfile)
    d = json.load(open(f'/content/drive/MyDrive/Results/bert_pp/{s}.json'),
                parse_float=lambda x: round(float(x), 6))
    with open(f'/content/drive/MyDrive/Results/bert_pp/{s}.json', "w") as outfile:
        json.dump(d, outfile)

len(err_pids), background = 0
[]
len(err_pids), method = 0
[]
len(err_pids), result = 0
[]


## Computing metric scores for TF-IDF, BERT - NLI, BERT - PP

In [ ]:
get_metric_scores('tf_idf', 'background', 20)

EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Total queries: 16; Total candidates: 1877
R-Precision: 0.1898
Precision@20: 0.2188
Recall@20: 0.3730
NDCG: 0.7178
NDCG@20: 0.4651
NDCG%20: 0.4669

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Total queries: 16; Total candidates: 1877
R-Precision: 0.1655
Precision@20: 0.2344
Recall@20: 0.3848
NDCG: 0.7346
NDCG@20: 0.4819
NDCG%20: 0.4922

AVERAGE METRICS
R-Precision: 0.1777
Precision@20: 0.2266
Recall@20: 0.3789
NDCG: 0.7262
NDCG@20: 0.4735
NDCG%20: 0.4795


In [ ]:
get_metric_scores('tf_idf', 'method', 20)

EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Total queries: 17; Total candidates: 2174
R-Precision: 0.0895
Precision@20: 0.0813
Recall@20: 0.2537
NDCG: 0.5544
NDCG@20: 0.2582
NDCG%20: 0.2574

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Total queries: 17; Total candidates: 2174
R-Precision: 0.0888
Precision@20: 0.0684
Recall@20: 0.2330
NDCG: 0.5335
NDCG@20: 0.2296
NDCG%20: 0.2306

AVERAGE METRICS
R-Precision: 0.0892
Precision@20: 0.0748
Recall@20: 0.2434
NDCG: 0.5439
NDCG@20: 0.2439
NDCG%20: 0.2440


In [ ]:
get_metric_scores('tf_idf', 'result', 20)

EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Total queries: 17; Total candidates: 2193
R-Precision: 0.1157
Precision@20: 0.1500
Recall@20: 0.3515
NDCG: 0.6769
NDCG@20: 0.4336
NDCG%20: 0.4347

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Total queries: 17; Total candidates: 2193
R-Precision: 0.1009
Precision@20: 0.1167
Recall@20: 0.2619
NDCG: 0.6081
NDCG@20: 0.3257
NDCG%20: 0.3355

AVERAGE METRICS
R-Precision: 0.1083
Precision@20: 0.1333
Recall@20: 0.3067
NDCG: 0.6425
NDCG@20: 0.3796
NDCG%20: 0.3851


In [ ]:
get_metric_scores('bert_nli', 'background', 20)

EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Total queries: 16; Total candidates: 1877
R-Precision: 0.2099
Precision@20: 0.2687
Recall@20: 0.4149
NDCG: 0.7735
NDCG@20: 0.5695
NDCG%20: 0.5731

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Total queries: 16; Total candidates: 1877
R-Precision: 0.1908
Precision@20: 0.2812
Recall@20: 0.4507
NDCG: 0.7734
NDCG@20: 0.5713
NDCG%20: 0.5831

AVERAGE METRICS
R-Precision: 0.2004
Precision@20: 0.2750
Recall@20: 0.4328
NDCG: 0.7735
NDCG@20: 0.5704
NDCG%20: 0.5781


In [ ]:
get_metric_scores('bert_nli', 'method', 20)

EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Total queries: 17; Total candidates: 2174
R-Precision: 0.1953
Precision@20: 0.1000
Recall@20: 0.3207
NDCG: 0.5974
NDCG@20: 0.3297
NDCG%20: 0.3299

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Total queries: 17; Total candidates: 2174
R-Precision: 0.1359
Precision@20: 0.1056
Recall@20: 0.3323
NDCG: 0.6139
NDCG@20: 0.3415
NDCG%20: 0.3488

AVERAGE METRICS
R-Precision: 0.1656
Precision@20: 0.1028
Recall@20: 0.3265
NDCG: 0.6056
NDCG@20: 0.3356
NDCG%20: 0.3393


In [ ]:
get_metric_scores('bert_nli', 'result', 20)

EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Total queries: 17; Total candidates: 2193
R-Precision: 0.1154
Precision@20: 0.1750
Recall@20: 0.3894
NDCG: 0.6493
NDCG@20: 0.3945
NDCG%20: 0.3922

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Total queries: 17; Total candidates: 2193
R-Precision: 0.1402
Precision@20: 0.1903
Recall@20: 0.4151
NDCG: 0.6583
NDCG@20: 0.4065
NDCG%20: 0.4222

AVERAGE METRICS
R-Precision: 0.1278
Precision@20: 0.1826
Recall@20: 0.4023
NDCG: 0.6538
NDCG@20: 0.4005
NDCG%20: 0.4072


In [ ]:
get_metric_scores('bert_pp', 'background', 20)

EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Total queries: 16; Total candidates: 1877
R-Precision: 0.2480
Precision@20: 0.3187
Recall@20: 0.5065
NDCG: 0.7650
NDCG@20: 0.5901
NDCG%20: 0.5852

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Total queries: 16; Total candidates: 1877
R-Precision: 0.2185
Precision@20: 0.3031
Recall@20: 0.4983
NDCG: 0.7870
NDCG@20: 0.6083
NDCG%20: 0.6096

AVERAGE METRICS
R-Precision: 0.2332
Precision@20: 0.3109
Recall@20: 0.5024
NDCG: 0.7760
NDCG@20: 0.5992
NDCG%20: 0.5974


In [ ]:
get_metric_scores('bert_pp', 'method', 20)

EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Total queries: 17; Total candidates: 2174
R-Precision: 0.2051
Precision@20: 0.0813
Recall@20: 0.2775
NDCG: 0.6290
NDCG@20: 0.3614
NDCG%20: 0.3762

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Total queries: 17; Total candidates: 2174
R-Precision: 0.1602
Precision@20: 0.1184
Recall@20: 0.4000
NDCG: 0.6410
NDCG@20: 0.3870
NDCG%20: 0.3969

AVERAGE METRICS
R-Precision: 0.1826
Precision@20: 0.0998
Recall@20: 0.3388
NDCG: 0.6350
NDCG@20: 0.3742
NDCG%20: 0.3865


In [ ]:
get_metric_scores('bert_pp', 'result', 20)

EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Total queries: 17; Total candidates: 2193
R-Precision: 0.1607
Precision@20: 0.2438
Recall@20: 0.5856
NDCG: 0.7148
NDCG@20: 0.5372
NDCG%20: 0.5388

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Total queries: 17; Total candidates: 2193
R-Precision: 0.1490
Precision@20: 0.2108
Recall@20: 0.5112
NDCG: 0.6949
NDCG@20: 0.4786
NDCG%20: 0.4978

AVERAGE METRICS
R-Precision: 0.1548
Precision@20: 0.2273
Recall@20: 0.5484
NDCG: 0.7048
NDCG@20: 0.5079
NDCG%20: 0.5183


In [ ]:
get_metric_scores('tf_idf', 'all', 20)

EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Total queries: 50; Total candidates: 6244
R-Precision: 0.1317
Precision@20: 0.1500
Recall@20: 0.3261
NDCG: 0.6497
NDCG@20: 0.3856
NDCG%20: 0.3863

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Total queries: 50; Total candidates: 6244
R-Precision: 0.1177
Precision@20: 0.1375
Recall@20: 0.2906
NDCG: 0.6225
NDCG@20: 0.3420
NDCG%20: 0.3489

AVERAGE METRICS
R-Precision: 0.1247
Precision@20: 0.1437
Recall@20: 0.3084
NDCG: 0.6361
NDCG@20: 0.3638
NDCG%20: 0.3676


In [ ]:
get_metric_scores('bert_nli', 'all', 20)

EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Total queries: 50; Total candidates: 6244
R-Precision: 0.1735
Precision@20: 0.1812
Recall@20: 0.3750
NDCG: 0.6734
NDCG@20: 0.4312
NDCG%20: 0.4317

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Total queries: 50; Total candidates: 6244
R-Precision: 0.1550
Precision@20: 0.1906
Recall@20: 0.3982
NDCG: 0.6803
NDCG@20: 0.4374
NDCG%20: 0.4490

AVERAGE METRICS
R-Precision: 0.1643
Precision@20: 0.1859
Recall@20: 0.3866
NDCG: 0.6768
NDCG@20: 0.4343
NDCG%20: 0.4404


In [ ]:
get_metric_scores('bert_pp', 'all', 20)

EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Total queries: 50; Total candidates: 6244
R-Precision: 0.2046
Precision@20: 0.2146
Recall@20: 0.4565
NDCG: 0.7029
NDCG@20: 0.4962
NDCG%20: 0.5001

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Total queries: 50; Total candidates: 6244
R-Precision: 0.1751
Precision@20: 0.2092
Recall@20: 0.4697
NDCG: 0.7058
NDCG@20: 0.4886
NDCG%20: 0.4989

AVERAGE METRICS
R-Precision: 0.1898
Precision@20: 0.2119
Recall@20: 0.4631
NDCG: 0.7043
NDCG@20: 0.4924
NDCG%20: 0.4995


## Bm25

In [ ]:
!pip install rank_bm25

In [ ]:
from rank_bm25 import BM25Okapi
import numpy as np
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import preprocess_documents, preprocess_string

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# bm25 all
# rectify changes before running
tokenized_corpus = list()
pid_list = list()
d = dict()
err_pids = list()
i = 0
for doc in docs:
    try:
        do = docs[doc].abstract
        if len(do) == 0:
            d[pid] = list()
            continue
        d[pid] = list()
        for string in do:
            d[pid] = d[pid] + preprocess_string(string)
        tokenized_corpus.append(d[pid])
        pid_list.append(pid)
    except:
        err_pids.append(pid)
print(f'len(err_pids) = {len(err_pids)}')
print(err_pids)

In [ ]:
bm25_index = BM25Okapi(tokenized_corpus)

In [ ]:
def search(search_string, num_results=len(tokenized_corpus)):
    search_tokens = preprocess_string(search_string)
    scores = bm25_index.get_scores(search_tokens)
    top_indexes = np.argsort(scores)[::-1][:num_results]
    for i in range(0, 10):
      print(scores[i])
    return top_indexes, scores

In [ ]:
indexes, scores = search(" ".join(get_data_by_facet('143814895', 'method')))

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


'We show that the actual diachronic frequency distributions and those expected under the neutral model do not coincide and conclude that in this case the neutral model does not provide an adequate description of change in ceramic decoration.'

## all-mpnet-base-v2

In [ ]:
# all_mpnet_base_v2 all
d = dict()
err_pids = list()
i = 0
for doc in docs:
    try:
        do = docs[doc].abstract
        if len(do) == 0:
            d[doc] = list()
            continue
        all_mpnet_base_v2_array = np.array(get_all_mpnet_base_v2_embedding(" ".join(do))).tolist()
        d[doc] = all_mpnet_base_v2_array
    except:
        err_pids.append(doc)
print(f'len(err_pids) = {len(err_pids)}')
print(err_pids)
with open("/content/drive/MyDrive/Results/all_mpnet_base_v2/all.json", "w") as outfile:
    json.dump(d, outfile)
d = json.load(open('/content/drive/MyDrive/Results/all_mpnet_base_v2/all.json'),
              parse_float=lambda x: round(float(x), 6))
with open('/content/drive/MyDrive/Results/all_mpnet_base_v2/all.json', "w") as outfile:
    json.dump(d, outfile)

len(err_pids) = 0
[]


In [ ]:
# all_mpnet_base_v2 queries
for i, ids in enumerate([background_query_ids, method_query_ids, result_query_ids]):
    d = dict()
    err_pids = list()
    s = ''
    if i == 0:
        s = 'background'
    elif i == 1:
        s = 'method'
    elif i == 2:
        s = 'result'
    for pid in ids:
        try:
            doc = get_data_by_facet(pid, s)
            if len(doc) == 0:
                d[pid] = list()
                continue
            all_mpnet_base_v2_query_vectors = np.array(get_all_mpnet_base_v2_embedding(" ".join(doc))).tolist()
            d[pid] = all_mpnet_base_v2_query_vectors
        except:
            err_pids.append(pid)
    print(f'len(err_pids), {s} = {len(err_pids)}')
    print(err_pids)
    with open(f'/content/drive/MyDrive/Results/all_mpnet_base_v2/{s}.json', "w") as outfile:
        json.dump(d, outfile)
    d = json.load(open(f'/content/drive/MyDrive/Results/all_mpnet_base_v2/{s}.json'),
                parse_float=lambda x: round(float(x), 6))
    with open(f'/content/drive/MyDrive/Results/all_mpnet_base_v2/{s}.json', "w") as outfile:
        json.dump(d, outfile)

len(err_pids), background = 0
[]
len(err_pids), method = 0
[]
len(err_pids), result = 0
[]


In [ ]:
get_metric_scores('all_mpnet_base_v2', 'background', 20)

EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Total queries: 16; Total candidates: 1877
R-Precision: 0.2920
Precision@20: 0.3500
Recall@20: 0.5772
NDCG: 0.8451
NDCG@20: 0.7066
NDCG%20: 0.7006

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Total queries: 16; Total candidates: 1877
R-Precision: 0.2673
Precision@20: 0.3438
Recall@20: 0.5729
NDCG: 0.8621
NDCG@20: 0.7209
NDCG%20: 0.7325

AVERAGE METRICS
R-Precision: 0.2797
Precision@20: 0.3469
Recall@20: 0.5750
NDCG: 0.8536
NDCG@20: 0.7137
NDCG%20: 0.7166


In [ ]:
get_metric_scores('all_mpnet_base_v2', 'result', 20)

EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Total queries: 17; Total candidates: 2193
R-Precision: 0.2552
Precision@20: 0.3188
Recall@20: 0.7516
NDCG: 0.8014
NDCG@20: 0.6742
NDCG%20: 0.6663

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Total queries: 17; Total candidates: 2193
R-Precision: 0.2443
Precision@20: 0.3038
Recall@20: 0.7037
NDCG: 0.7784
NDCG@20: 0.6309
NDCG%20: 0.6330

AVERAGE METRICS
R-Precision: 0.2498
Precision@20: 0.3113
Recall@20: 0.7276
NDCG: 0.7899
NDCG@20: 0.6526
NDCG%20: 0.6497


In [ ]:
get_metric_scores('all_mpnet_base_v2', 'method', 20)

EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Total queries: 17; Total candidates: 2174
R-Precision: 0.2313
Precision@20: 0.1688
Recall@20: 0.4829
NDCG: 0.6519
NDCG@20: 0.4378
NDCG%20: 0.4365

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Total queries: 17; Total candidates: 2174
R-Precision: 0.1697
Precision@20: 0.1594
Recall@20: 0.4640
NDCG: 0.6746
NDCG@20: 0.4659
NDCG%20: 0.4723

AVERAGE METRICS
R-Precision: 0.2005
Precision@20: 0.1641
Recall@20: 0.4734
NDCG: 0.6633
NDCG@20: 0.4518
NDCG%20: 0.4544


In [ ]:
get_metric_scores('all_mpnet_base_v2', 'all', 20)

EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Total queries: 50; Total candidates: 6244
R-Precision: 0.2595
Precision@20: 0.2792
Recall@20: 0.6039
NDCG: 0.7662
NDCG@20: 0.6062
NDCG%20: 0.6012

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Total queries: 50; Total candidates: 6244
R-Precision: 0.2262
Precision@20: 0.2675
Recall@20: 0.5800
NDCG: 0.7698
NDCG@20: 0.6034
NDCG%20: 0.6099

AVERAGE METRICS
R-Precision: 0.2429
Precision@20: 0.2733
Recall@20: 0.5919
NDCG: 0.7680
NDCG@20: 0.6048
NDCG%20: 0.6055


In [ ]:
get_metric_scores('alberta', 'background', 20)

EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Total queries: 16; Total candidates: 1877
R-Precision: 0.2723
Precision@20: 0.2812
Recall@20: 0.3972
NDCG: 0.7780
NDCG@20: 0.5871
NDCG%20: 0.5875

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Total queries: 16; Total candidates: 1877
R-Precision: 0.2297
Precision@20: 0.2844
Recall@20: 0.4266
NDCG: 0.7839
NDCG@20: 0.5917
NDCG%20: 0.6027

AVERAGE METRICS
R-Precision: 0.2510
Precision@20: 0.2828
Recall@20: 0.4119
NDCG: 0.7809
NDCG@20: 0.5894
NDCG%20: 0.5951


In [ ]:
get_metric_scores('alberta', 'result', 20)

EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Total queries: 17; Total candidates: 2193
R-Precision: 0.1694
Precision@20: 0.2313
Recall@20: 0.5335
NDCG: 0.7087
NDCG@20: 0.5165
NDCG%20: 0.5097

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Total queries: 17; Total candidates: 2193
R-Precision: 0.1513
Precision@20: 0.1906
Recall@20: 0.4340
NDCG: 0.6731
NDCG@20: 0.4415
NDCG%20: 0.4511

AVERAGE METRICS
R-Precision: 0.1603
Precision@20: 0.2109
Recall@20: 0.4837
NDCG: 0.6909
NDCG@20: 0.4790
NDCG%20: 0.4804


In [ ]:
get_metric_scores('alberta', 'method', 20)

EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Total queries: 17; Total candidates: 2174
R-Precision: 0.1364
Precision@20: 0.0875
Recall@20: 0.2932
NDCG: 0.5832
NDCG@20: 0.3019
NDCG%20: 0.3056

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Total queries: 17; Total candidates: 2174
R-Precision: 0.1207
Precision@20: 0.1215
Recall@20: 0.4204
NDCG: 0.6156
NDCG@20: 0.3594
NDCG%20: 0.3637

AVERAGE METRICS
R-Precision: 0.1285
Precision@20: 0.1045
Recall@20: 0.3568
NDCG: 0.5994
NDCG@20: 0.3307
NDCG%20: 0.3346


In [ ]:
get_metric_scores('alberta', 'all', 20)

EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Total queries: 50; Total candidates: 6244
R-Precision: 0.1927
Precision@20: 0.2000
Recall@20: 0.4079
NDCG: 0.6900
NDCG@20: 0.4685
NDCG%20: 0.4676

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Total queries: 50; Total candidates: 6244
R-Precision: 0.1663
Precision@20: 0.1971
Recall@20: 0.4268
NDCG: 0.6890
NDCG@20: 0.4616
NDCG%20: 0.4698

AVERAGE METRICS
R-Precision: 0.1795
Precision@20: 0.1986
Recall@20: 0.4174
NDCG: 0.6895
NDCG@20: 0.4650
NDCG%20: 0.4687


In [ ]:
model_sent_distbert_nli = SentenceTransformer('all-distilroberta-v1')

In [ ]:
model_alberta = SentenceTransformer('paraphrase-albert-small-v2')

## DISTILBERT base model

In [ ]:
# distil_bert_nli all
d = dict()
err_pids = list()
i = 0
for doc in docs:
    try:
        do = docs[doc].abstract
        if len(do) == 0:
            d[doc] = []
            continue
        distilbert_nli_vectors = np.array(get_distilbert_base_v2_embedding(" ".join(do))).tolist()
        d[doc] = distilbert_nli_vectors
    except:
        err_pids.append(doc)
        print("error")
print(f'len(err_pids) = {len(err_pids)}')
print(err_pids)
with open("/content/drive/MyDrive/Results/bert_nli/tuned_all.json", "w") as outfile:
    json.dump(d, outfile)
d = json.load(open('/content/drive/MyDrive/Results/bert_nli/tuned_all.json'),
              parse_float=lambda x: round(float(x), 6))
with open('/content/drive/MyDrive/Results/bert_nli/tuned_all.json', "w") as outfile:
    json.dump(d, outfile)

In [ ]:
# distil_bert_nli queries
for i, ids in enumerate([background_query_ids, method_query_ids, result_query_ids]):
    d = dict()
    err_pids = list()
    s = ''
    if i == 0:
        s = 'background'
    elif i == 1:
        s = 'method'
    elif i == 2:
        s = 'result'
    for pid in ids:
        try:
            doc = get_data_by_facet(pid, s)
            if len(doc) == 0:
                d[pid] = []
                continue
            distilbert_nli_query_vectors = np.array(get_distilbert_base_v2_embedding(" ".join(doc))).tolist()
            d[pid] = distilbert_nli_query_vectors
        except:
            err_pids.append(pid)
    print(f'len(err_pids), {s} = {len(err_pids)}')
    print(err_pids)
    with open(f'/content/drive/MyDrive/Results/bert_nli/tuned_{s}.json', "w") as outfile:
        json.dump(d, outfile)
    d = json.load(open(f'/content/drive/MyDrive/Results/bert_nli/tuned_{s}.json'),
                parse_float=lambda x: round(float(x), 6))
    with open(f'/content/drive/MyDrive/Results/bert_nli/tuned_{s}.json', "w") as outfile:
        json.dump(d, outfile)

In [ ]:
get_metric_scores('distilbert_nli', 'background', 20)

In [ ]:
get_metric_scores('distilbert_nli', 'method', 20)

In [ ]:
get_metric_scores('distilbert_nli', 'result', 20)

In [ ]:
get_metric_scores('distilbert_nli', 'result', 20)

In [ ]:
get_metric_scores('distilbert_nli', 'all', 20)

## ALBERT

In [ ]:
# alberta all
d = dict()
err_pids = list()
i = 0
for doc in docs:
    try:
        do = docs[doc].abstract
        if len(do) == 0:
            d[doc] = []
            continue
        distilbert_nli_vectors = np.array(get_alberta_embedding(" ".join(do))).tolist()
        d[doc] = distilbert_nli_vectors
    except:
        err_pids.append(doc)
        print("error")
print(f'len(err_pids) = {len(err_pids)}')
print(err_pids)
with open("/content/drive/MyDrive/Results/alberta/all.json", "w") as outfile:
    json.dump(d, outfile)
d = json.load(open('/content/drive/MyDrive/Results/alberta/all.json'),
              parse_float=lambda x: round(float(x), 6))
with open('/content/drive/MyDrive/Results/alberta/all.json', "w") as outfile:
    json.dump(d, outfile)

In [ ]:
# alberta queries
for i, ids in enumerate([background_query_ids, method_query_ids, result_query_ids]):
    d = dict()
    err_pids = list()
    s = ''
    if i == 0:
        s = 'background'
    elif i == 1:
        s = 'method'
    elif i == 2:
        s = 'result'
    for pid in ids:
        try:
            doc = get_data_by_facet(pid, s)
            if len(doc) == 0:
                d[pid] = []
                continue
            distilbert_nli_query_vectors = np.array(get_alberta_embedding(" ".join(doc))).tolist()
            d[pid] = distilbert_nli_query_vectors
        except:
            err_pids.append(pid)
    print(f'len(err_pids), {s} = {len(err_pids)}')
    print(err_pids)
    with open(f'/content/drive/MyDrive/Results/alberta/{s}.json', "w") as outfile:
        json.dump(d, outfile)
    d = json.load(open(f'/content/drive/MyDrive/Results/alberta/{s}.json'),
                parse_float=lambda x: round(float(x), 6))
    with open(f'/content/drive/MyDrive/Results/alberta/{s}.json', "w") as outfile:
        json.dump(d, outfile)

In [ ]:
get_metric_scores('alberta', 'background', 20)

In [ ]:
get_metric_scores('alberta', 'method', 20)

In [ ]:
get_metric_scores('alberta', 'result', 20)

In [ ]:
get_metric_scores('alberta', 'all', 20)

## ALLENAI - SPECTER

In [ ]:
specter_tokenize = AutoTokenizer.from_pretrained('allenai/specter')
specter_model = AutoModel.from_pretrained('allenai/specter')

In [ ]:
# for all
d = dict()
err_pids = list()
i = 0
cnt = 0;
for doc in docs:
    try:
        do = docs[doc].abstract
        if len(do) == 0:
            d[doc] = list()
            continue
        get_specter = get_allenai_specter_embedding(" ".join(do)).detach().numpy().tolist()
        cnt += 1
        print(cnt)
        d[doc] = get_specter[0]
    except:
        err_pids.append(doc)
print(f'len(err_pids) = {len(err_pids)}')
print(err_pids)
with open('/content/drive/MyDrive/Results/allenai_specter/all.json', 'w') as outfile:
    json.dump(d, outfile)
d = json.load(open('/content/drive/MyDrive/Results/allenai_specter/all.json'),
              parse_float=lambda x: round(float(x), 6))
with open('/content/drive/MyDrive/Results/allenai_specter/all.json', 'w') as outfile:
    json.dump(d, outfile)

In [ ]:
# specter queries
for i, ids in enumerate([background_query_ids, method_query_ids, result_query_ids]):
    d = dict()
    err_pids = list()
    s = ''
    if i == 0:
        s = 'background'
    elif i == 1:
        s = 'method'
    elif i == 2:
        s = 'result'
    for pid in ids:
        try:
            doc = get_data_by_facet(pid, s)
            if len(doc) == 0:
                d[pid] = list()
                continue
            specter_array = get_allenai_specter_embedding(" ".join(doc)).detach().numpy().tolist()
            d[pid] = specter_array[0]
        except:
            err_pids.append(pid)
    print(f'len(err_pids), {s} = {len(err_pids)}')
    print(err_pids)
    with open(f'/content/drive/MyDrive/Results/allenai_specter/{s}.json', 'w') as outfile:
        json.dump(d, outfile)
    d = json.load(open(f'/content/drive/MyDrive/Results/allenai_specter/{s}.json'),
                parse_float=lambda x: round(float(x), 6))
    with open(f'/content/drive/MyDrive/Results/allenai_specter/{s}.json', 'w') as outfile:
        json.dump(d, outfile)

In [ ]:
get_metric_scores('allenai_specter', 'background', 20)

In [ ]:
get_metric_scores('allenai_specter', 'method', 20)

In [ ]:
get_metric_scores('allenai_specter', 'result', 20)

In [ ]:
get_metric_scores('allenai_specter', 'all', 20)